In [9]:
import pandas as pd
import requests
import time
import re
from bs4 import BeautifulSoup
from transformers import pipeline
from typing import Dict, Any

# Define SEC EDGAR API headers
HEADERS = {'User-Agent': 'BCG-FinancialBot'}

# Step 1: Get the 10-K Filing URL
def get_10k_filing_url(company_cik, year):
    search_url = f"https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={company_cik}&type=10-K&dateb={year}1231&owner=exclude&count=10"
    response = requests.get(search_url, headers=HEADERS)

    if response.status_code != 200:
        print(f"Failed to retrieve filings for {company_cik} in {year}.")
        return None

    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract the first 10-K document link
    for link in soup.find_all('a', href=True):
        if 'Archives/edgar/data' in link['href']:
            filing_page_url = "https://www.sec.gov" + link['href']
            return filing_page_url

    return None

# Step 2: Extract Financial Data from 10-K Report
def extract_financial_data(filing_url):
    response = requests.get(filing_url, headers=HEADERS)
    if response.status_code != 200:
        print(f"Failed to fetch 10-K filing from {filing_url}")
        return None

    soup = BeautifulSoup(response.text, 'html.parser')
    text = soup.get_text()

    financial_data = {}

    # Extract financial metrics using regex patterns
    patterns = {
        "Total Revenue": r"Total\s+Revenue.*?(\d{1,3}(?:,\d{3})*(?:\.\d+)?)",
        "Net Income": r"Net\s+Income.*?(\d{1,3}(?:,\d{3})*(?:\.\d+)?)",
        "Total Assets": r"Total\s+Assets.*?(\d{1,3}(?:,\d{3})*(?:\.\d+)?)",
        "Total Liabilities": r"Total\s+Liabilities.*?(\d{1,3}(?:,\d{3})*(?:\.\d+)?)",
        "Cash Flow from Operating Activities": r"Cash\s+Flow\s+from\s+Operating\s+Activities.*?(\d{1,3}(?:,\d{3})*(?:\.\d+)?)"
    }

    for key, pattern in patterns.items():
        match = re.search(pattern, text, re.IGNORECASE)
        financial_data[key] = match.group(1) if match else "Not Found"

    return financial_data

# Step 3: Extract CEO's Letter or MD&A Section for Sentiment Analysis
def extract_ceo_statement(filing_url):
    response = requests.get(filing_url, headers=HEADERS)
    if response.status_code != 200:
        return "Not Found"

    soup = BeautifulSoup(response.text, 'html.parser')
    text = soup.get_text()

    # Extract MD&A section (longer text for sentiment analysis)
    mdna_match = re.search(r"(Management’s Discussion and Analysis.*?Item\s+\d+)", text, re.DOTALL | re.IGNORECASE)

    return mdna_match.group(1)[:2000] if mdna_match else "Not Found"  # Extract first 2000 characters

# Step 4: Perform Sentiment Analysis using FinBERT
sentiment_pipeline = pipeline("sentiment-analysis", model="ProsusAI/finbert")

def analyze_sentiment(text):
    if text == "Not Found":
        return "Neutral"

    sentiment_result = sentiment_pipeline(text[:512])  # Analyze first 512 characters
    return sentiment_result[0]['label']

# Step 5: Aggregate Financial Data
def analyze_financial_trends(financial_data_list: list[Dict[str, Any]]) -> pd.DataFrame:
    if not financial_data_list:
        print("No financial data found.")
        return pd.DataFrame()

    df = pd.DataFrame(financial_data_list)

    if 'Company' not in df.columns or 'Year' not in df.columns:
        print("Missing Company or Year columns.")
        return pd.DataFrame()

    df.set_index(["Company", "Year"], inplace=True)

    # Convert financial values to numeric
    for col in ["Total Revenue", "Net Income", "Total Assets", "Total Liabilities", "Cash Flow from Operating Activities"]:
        df[col] = df[col].replace({",": ""}, regex=True)  # Remove commas
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # Calculate Year-over-Year Growth
    df["Revenue Growth (%)"] = df.groupby("Company")["Total Revenue"].pct_change() * 100
    df["Net Income Growth (%)"] = df.groupby("Company")["Net Income"].pct_change() * 100

    return df

# Step 6: Main Execution
companies = {'Microsoft': '789019', 'Tesla': '1318605', 'Apple': '320193'}
years = ['2023', '2022', '2021']

data_collection = []
for company, cik in companies.items():
    for year in years:
        print(f"Fetching {company} 10-K filing for {year}...")
        filing_url = get_10k_filing_url(cik, year)

        if filing_url:
            financial_data = extract_financial_data(filing_url)
            ceo_statement = extract_ceo_statement(filing_url)
            financial_data['Company'] = company
            financial_data['Year'] = year
            financial_data['CEO Sentiment'] = analyze_sentiment(ceo_statement)
            data_collection.append(financial_data)

        time.sleep(1)  # Prevent hitting SEC rate limits

# Convert to DataFrame and Analyze Trends
financial_df = analyze_financial_trends(data_collection)
if not financial_df.empty:
    print(financial_df)
    financial_df.to_csv("financial_analysis.csv")
else:
    print("No financial analysis data to save.")


Device set to use cpu


Fetching Microsoft 10-K filing for 2023...
Fetching Microsoft 10-K filing for 2022...
Fetching Microsoft 10-K filing for 2021...
Fetching Tesla 10-K filing for 2023...
Fetching Tesla 10-K filing for 2022...
Fetching Tesla 10-K filing for 2021...
Fetching Apple 10-K filing for 2023...
Fetching Apple 10-K filing for 2022...
Fetching Apple 10-K filing for 2021...
                Total Revenue  Net Income  Total Assets  Total Liabilities  \
Company   Year                                                               
Microsoft 2023            NaN         NaN           NaN                NaN   
          2022            NaN         NaN           NaN                NaN   
          2021            NaN         NaN           NaN                NaN   
Tesla     2023            NaN         NaN           NaN                NaN   
          2022            NaN         NaN           NaN                NaN   
          2021            NaN         NaN           NaN                NaN   
Apple     202

<ipython-input-9-59ec06461b1d>:102: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df["Revenue Growth (%)"] = df.groupby("Company")["Total Revenue"].pct_change() * 100
<ipython-input-9-59ec06461b1d>:103: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df["Net Income Growth (%)"] = df.groupby("Company")["Net Income"].pct_change() * 100
